In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import os

from keras.layers.core import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [14]:
# Setup
path = './data/hmdb51'
path_rowframes = './data/hmdb51/rawframes/'
path_annotations = './data/hmdb51/annotations/'

img_height = 224
img_width = 224
batch_size = 32
num_classes = 51

In [6]:
num_frames_desired = 17     #number of frames per clip
type_frame = 'flow'          #img / flow_x / flow_y
partition = 'train'

In [7]:
def frame_selector(list_frames, opt_flow_len, num_of_snip):
    total_frames = len(list_frames)
    win_len = (total_frames - opt_flow_len) // num_of_snip
    start_frame = int(random.random() * win_len) + 1
    for i in range(num_of_snip):
        frames += range(start_frame + opt_flow_len * i, start_frame + opt_flow_len * (i + 1))
    return sel_frames

In [12]:

def sampling(list, num_frames_desired):
    step = len(list) // (num_frames_desired)
    #selected_frames = list(range(0, len(list), step))[:num_frames_desired]
    sampled_list = list[0:len(list):step][:num_frames_desired]
    return(sampled_list)

def find_paths(partition, type_frame, num_frames_desired):
    if partition == 'train':
        video_list = pd.read_csv(f'{path_annotations}/hmdb51_train_split_1_rawframes.txt', sep=" ", header=None) #train
        video_list.columns = ["path", "num_frames_tot", "class"]
    elif partition == 'val':
        video_list = pd.read_csv(f'{path_annotations}/hmdb51_val_split_1_rawframes.txt', sep=" ", header=None) #test
        video_list.columns = ["path", "num_frames_tot", "class"]
    else:
        raise Exception("invalid partition")

    #temp_path = video_list.loc[0]['path'] #da togliere!!! -> PATH CARTELLA
    # qua shaffolare le righe del dataframe
    paths = []
    classes = []
    for index, row in video_list.iterrows(): #da togliere [:1]
        temp_path = row['path']                    #da rimuovere il commentato
        frame_list = os.listdir(os.path.join(f'./{temp_path}')) # lista dei file nella cartella del video

        frame_list_type = [i for i in frame_list if i.startswith(f'{type_frame}')] # selezionare gli optical flow o le immagini (solo quelli contenenti prefisso)

        filename = frame_selector(frame_list_type, num_frames_desired) # ATTENZIONE MODIFICA

        paths.extend([os.path.join('.\\', temp_path, file) for file in filename]) # aggiungo .// al path del video al nome del frame
        temp = [row['class']] * num_frames_desired # replica la classe su tutti i frame
        classes.extend(temp) # non è un append, unica lista

    return(list(zip(paths, classes))) # ritornati i percorsi dei file 

def find_videos_and_metadata(partition):
    if partition == 'train':
        video_list = pd.read_csv(f'{path_annotations}/hmdb51_train_split_1_rawframes.txt', sep=" ", header=None) #train
        video_list.columns = ["path", "num_frames_tot", "class"]
    elif partition == 'val':
        video_list = pd.read_csv(f'{path_annotations}/hmdb51_val_split_1_rawframes.txt', sep=" ", header=None) #test
        video_list.columns = ["path", "num_frames_tot", "class"]
    else:
        raise Exception("invalid partition")
    return(video_list)

In [14]:
video_list = find_videos_and_metadata('train')
video_list

,path,num_frames_tot,class
0,data\hmdb51\rawframes\brush_hair\April_09_brus...,407,0
1,data\hmdb51\rawframes\brush_hair\April_09_brus...,393,0
2,data\hmdb51\rawframes\brush_hair\April_09_brus...,321,0
3,data\hmdb51\rawframes\brush_hair\Aussie_Brunet...,157,0
4,data\hmdb51\rawframes\brush_hair\Aussie_Brunet...,135,0
...,...,...,...
3565,data\hmdb51\rawframes\wave\prideandprejudice1_...,83,50
3566,data\hmdb51\rawframes\wave\veoh_harold_and_kum...,47,50
3567,data\hmdb51\rawframes\wave\wave_and_say_hi_wav...,94,50
3568,data\hmdb51\rawframes\wave\winKen_wave_u_cm_np...,41,50


In [9]:
def parse_image(filename):
    image = tf.io.read_file(filename) 
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    return image

def configure_for_performance(ds):
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.repeat()
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

In [10]:
# ------- create train set
# Lista di tuple [[path_/classe]]
filenames = find_paths(partition='train', type_frame=type_frame, num_frames_desired=num_frames_desired)

# random.shuffle(filenames)

zipped = [list(t) for t in zip(*filenames)]

filenames = zipped[0]
labels = zipped[1]

filenames_ds = tf.data.Dataset.from_tensor_slices(filenames) # trasforma nel tensore

images_ds = filenames_ds.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE) #
labels_ds = tf.data.Dataset.from_tensor_slices(labels)
ds = tf.data.Dataset.zip((images_ds, labels_ds))
train_ds = configure_for_performance(ds)

NameError: name 'pd' is not defined

# Motion Stream Model Definition

In [31]:
checkpoint_filepath = './Models/motion_model{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_sparse_categorical_accuracy',
    mode='max',
    save_best_only=True)

In [32]:
num_classes = 51

model_mot= keras.models.Sequential()

# data_augmentation

model_mot.add(keras.layers.Conv2D(96, (7,7), strides = 2, input_shape=(224, 224, 20), activation = "relu"))
model_mot.add(keras.layers.BatchNormalization())
model_mot.add(keras.layers.MaxPooling2D((3,3), strides=2, padding="same"))

model_mot.add(keras.layers.ZeroPadding2D(padding = (1,1)))
model_mot.add(keras.layers.Conv2D(256, (5,5), strides = 2, activation='relu'))
model_mot.add(keras.layers.BatchNormalization())
model_mot.add(keras.layers.MaxPooling2D((3,3), strides=2, padding="same"))
          
model_mot.add(keras.layers.ZeroPadding2D(padding = (1,1)))
model_mot.add(keras.layers.Conv2D(512, (3,3), strides = 1, activation='relu'))

model_mot.add(keras.layers.ZeroPadding2D(padding = (1,1)))
model_mot.add(keras.layers.Conv2D(512, (3,3), strides = 1, activation='relu'))

model_mot.add(keras.layers.ZeroPadding2D(padding = (1,1)))
model_mot.add(keras.layers.Conv2D(512, (3,3), strides = 1, activation='relu'))
model_mot.add(keras.layers.MaxPooling2D((3,3), strides=2, padding="same"))

model_mot.add(keras.layers.Flatten())

model_mot.add(keras.layers.Dense(4096, activation='relu'))
model_mot.add(keras.layers.Dropout(0.5))

model_mot.add(keras.layers.Dense(2048, activation='relu'))
model_mot.add(keras.layers.Dropout(0.5))

model_mot.add(keras.layers.Dense(num_classes, activation="softmax"))

In [4]:
model_mot.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 109, 109, 96)      94176     
                                                                 
 batch_normalization (BatchN  (None, 109, 109, 96)     384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 55, 55, 96)       0         
 )                                                               
                                                                 
 zero_padding2d (ZeroPadding  (None, 57, 57, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                        

In [33]:
optimizer = keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, nesterov=True)
# keras.optimizers.Adam(learning_rate=0.001)

model_mot.compile(loss=keras.losses.sparse_categorical_crossentropy, 
                   metrics=['sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'], 
                   optimizer=optimizer)

# Batch Generation Setting

In [34]:
# Setup
path = './data/hmdb51'
path_rowframes = './data/hmdb51/rawframes/'
path_annotations = './data/hmdb51/annotations/'

img_height = 224
img_width = 224
batch_size = 32
num_classes = 51

# Parametri del batch generator
num_of_snip=1
opt_flow_len=10
image_shape=(img_height, img_width)

In [44]:
class DataSet():
    def __init__(self, 
                 num_of_snip=1, 
                 opt_flow_len=10, 
                 image_shape=(224, 224),
                 partition='train'):
        
    # opt_flow_len = (int) number of optical flow frames pet stacked optical flow (snip)

        self.opt_flow_len = opt_flow_len
        self.num_of_snip = num_of_snip
        self.image_shape = image_shape
        self.opt_flow_path = os.path.join(path_rowframes)
        self.path_annotations = path_annotations
        self.partition = partition
        
        # Get data
        self.video_list = self.find_videos_and_metadata()
        
    def find_videos_and_metadata(self):
        if self.partition == 'train':
            video_list = pd.read_csv(f'{self.path_annotations}/hmdb51_train_split_1_rawframes.txt', sep=" ", header=None) #train
            video_list.columns = ["path", "num_frames_tot", "class"]
        elif self.partition == 'val':
            video_list = pd.read_csv(f'{self.path_annotations}/hmdb51_val_split_1_rawframes.txt', sep=" ", header=None) #test
            video_list.columns = ["path", "num_frames_tot", "class"]
        else:
            raise Exception("invalid partition")
        return(video_list)
    
    def generator(self, batch_size):
        video_list = self.video_list
        idx = 0
        print(f"Creating {self.partition} with {len(self.video_list)} samples.")
        while 1:
            idx +=1
            #print(f"Generator creating batch {idx}")
            X, y = [], []
            for _ in range(batch_size):
                # Reset 
                stack = []

                # Get a random sample.
                row = video_list.sample(n=1).values.tolist()

                # Get the stacked optical flows from disk.
                stack = self.find_stacked_optical_flows(row)

                X.append(stack)
                y.append(row[0][2])

            X = np.array(X)
            y = np.array(y)
            y = np.squeeze(y)

            yield X, y
        
    def generator1(self, batch_size):
        video_list = self.video_list
        idx = 0
        print(f"Creating {self.partition} with {len(self.video_list)} samples.")
        idx +=1
        #print(f"Generattor creating batch {idx}")
        X, y = [], []
        for _ in range(batch_size):
            # Reset 
            stack = []

            # Get a random sample.
            row = video_list.sample(n=1).values.tolist()

            # Get the stacked optical flows from disk.
            stack = self.find_stacked_optical_flows(row)
            print(stack)
            X.append(stack)
            y.append(row[0][2])

        X = np.array(X)
        y = np.array(y)
        y = np.squeeze(y)

        return X, y
    
            
    def find_stacked_optical_flows(self, row):
        opt_flow_stack = []
        opt_flow_dir = self.opt_flow_path
        
        # Temporal parameters
        total_frames = row[0][1]
        win_len = (total_frames - self.opt_flow_len) // self.num_of_snip
        if self.partition=='train':
            start_frame = int(random.random() * win_len) + 1
        else:
            start_frame = int(0.5 * win_len) + 1
        frames = [] # selected optical flow frames
        for i in range(self.num_of_snip):
            frames += range(start_frame + self.opt_flow_len * i, 
                            start_frame + self.opt_flow_len * (i + 1))  
        if self.partition == 'train' and random.random() > 0.5:
            flip = True
        else:
            flip = False
        
        # Spatial Parameter
        img_test = cv2.imread(os.path.join(f'.\\{row[0][0]}' + '\\flow_x_' + str("%05d"%(1)) + '.jpg'), 0)
        top = int((img_test.shape[0] - self.image_shape[0]) * random.random())
        left = int((img_test.shape[1] - self.image_shape[1]) * random.random())
        right = left + self.image_shape[1]
        bottom = top + self.image_shape[0]
        
        # loop over frames
        for i_frame in frames:
            # x flow
            img = None # reset to be safe
            temp_path = None
            temp_path = row[0][0]
            img = cv2.imread(os.path.join(f'.\\{temp_path}' + '\\flow_x_' + str("%05d"%(i_frame)) + '.jpg'), 0)
            #print(os.path.join(f'.\\{temp_path}' + '\\flow_x_' + str("%05d"%(i_frame)) + '.jpg'))
            #print(img.shape)
            img = np.array(img)
            # mean substraction 
            img = img - np.mean(img)
            if self.partition == 'train':
                # random crop
                img = img[top : bottom, left : right]
            else:
                #resize
                img = cv2.resize(img, self.image_shape)
            img = img / 255. # normalize pixels 
            if flip:
                img = -img
            img = cv2.resize(img, self.image_shape)
            #print(img.shape)
            opt_flow_stack.append(img)
            
            # y flow
            img2 = None # reset to be safe
            img2 = cv2.imread(os.path.join(f'.\\{temp_path}' + '\\flow_y_' + str("%05d"%(i_frame)) + '.jpg'), 0)
            #print(img2.shape)
            img2 = np.array(img2)
            #img2 = np.swapaxes(img2, 0, 1)
            img2 = img2 - np.mean(img2)
            if self.partition == 'train':
                 #random crop
                img2 = img2[top : bottom, left : right]
            else:
                #resize
                img2= cv2.resize(img2, self.image_shape)
            img2 = img2 / 255. # normalize pixels 
            if flip:
                img2 = -img2
            img2 = cv2.resize(img2, self.image_shape)
            #print(img2.shape)
            opt_flow_stack.append(img2)
            
        opt_flow_stack = np.array(opt_flow_stack)
        opt_flow_stack = np.swapaxes(opt_flow_stack, 0, 1)
        opt_flow_stack = np.swapaxes(opt_flow_stack, 1, 2)
        # random horizontal flip for training sets
        if flip:
            opt_flow_stack = np.flip(opt_flow_stack, 0)
        return opt_flow_stack
            
        

# Model Fit

Test

In [45]:
videos_train = DataSet(num_of_snip=num_of_snip, opt_flow_len=opt_flow_len, partition='train')
videos_val = DataSet(num_of_snip=num_of_snip, opt_flow_len=opt_flow_len, partition='val')

In [9]:
X, y = videos_train.generator1(32)
X.shape

Creating train with 3570 samples.
Generattor creating batch 1
[[[ 2.53594771e-03  1.21383272e-02 -1.49873366e-02 ... -3.95174632e-03
    4.56265319e-03 -9.84160539e-03]
  [ 2.53594771e-03  1.21383272e-02 -1.49873366e-02 ... -3.95174632e-03
    4.56265319e-03 -9.84160539e-03]
  [ 2.53594771e-03  1.21383272e-02 -1.49873366e-02 ... -3.95174632e-03
    4.56265319e-03 -9.84160539e-03]
  ...
  [ 1.03790850e-02  4.29518995e-03 -7.14419935e-03 ... -3.95174632e-03
    4.56265319e-03  1.92310049e-03]
  [ 1.03790850e-02  4.29518995e-03 -7.14419935e-03 ... -3.95174632e-03
    4.56265319e-03  1.92310049e-03]
  [ 2.53594771e-03  4.29518995e-03 -7.14419935e-03 ... -3.95174632e-03
    4.56265319e-03  1.92310049e-03]]

 [[ 2.53594771e-03  1.60598958e-02 -1.49873366e-02 ... -3.95174632e-03
    4.56265319e-03 -9.84160539e-03]
  [ 2.53594771e-03  1.60598958e-02 -1.49873366e-02 ... -3.95174632e-03
    4.56265319e-03 -9.84160539e-03]
  [ 2.53594771e-03  1.60598958e-02 -1.49873366e-02 ... -3.95174632e-03
   

(32, 224, 224, 20)

Fit

In [46]:
# batch size 
step_per_epoch_train = len(videos_train.video_list) // batch_size
print(step_per_epoch_train)
step_per_epoch_val = len(videos_val.video_list) // batch_size
print(step_per_epoch_val)

111
47


In [47]:
train_generator = videos_train.generator(batch_size)

val_generator = videos_val.generator(batch_size)

In [48]:
# %tb
history_cnn_spatial_stream = model_mot.fit_generator(generator =train_generator, 
                                            steps_per_epoch=step_per_epoch_train, 
                                            validation_data=val_generator,
                                            validation_steps=step_per_epoch_val, 
                                            epochs=20,
                                            callbacks=[model_checkpoint_callback])
                

c:\Users\giorg\Documents\venv\deepL37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Creating train with 3570 samples.
Epoch 1/20
111/111 [==============================] - 317s 3s/step - loss: 3.9174 - sparse_categorical_accuracy: 0.0310 - sparse_top_k_categorical_accuracy: 0.1188 - val_loss: 3.9260 - val_sparse_categorical_accuracy: 0.0180 - val_sparse_top_k_categorical_accuracy: 0.1097
Epoch 2/20
111/111 [==============================] - 266s 2s/step - loss: 3.8799 - sparse_categorical_accuracy: 0.0408 - sparse_top_k_categorical_accuracy: 0.1537 - val_loss: 3.9207 - val_sparse_categorical_accuracy: 0.0273 - val_sparse_top_k_categorical_accuracy: 0.1090
Epoch 3/20
111/111 [==============================] - 501s 5s/step - loss: 3.8310 - sparse_categorical_accuracy: 0.0448 - sparse_top_k_categorical_accuracy: 0.1788 - val_loss: 4.0949 - val_sparse_categorical_accuracy: 0.0233 - val_sparse_top_k_categorical_accuracy: 0.1130
Epoch 4/20
111/111 [==============================] - 479s 4s/step - loss: 3.8021 - sparse_categorical_accuracy: 0.0467 - sparse_top_k_categorical_

In [ ]:
score = model_mot.evaluate_generator(val_generator, verbose=1, batch_size = batch_size, steps=step_per_epoch_val)
print('sparse_categorical_accuracy:', score[0])
print('sparse_top_k_categorical_accuracy:', score[1])

In [ ]:
model_mot.save('./Models/model_mot.h5')